In [1]:
import pandas as pd
import numpy as np

In [2]:
!pwd

/home/dileep/Desktop/python/QuickUMLS-master


In [3]:
# Importing data
txt = pd.read_csv('/home/dileep/Desktop/txt.csv')
txt = txt.loc[:,'Unnamed: 0']
sample2 = txt[3]

In [5]:
sample2

' Pt has history of fall from cycle. he is a bit disoriented and gcs 12/15 CT shows subdural hematoma with midline shift. Neurosurgery review to be done CNS: Urgent neurosurgery review to be done in view of the midline shift and subdural hematoma in the CT report. GCS 12/15. No information regarding LOC.  If GCS drops more,then plan a repeat CT scan. Will start anti-edema measures with Mannitol 100 ml TID. Start Inj. levetiracetam 500 mg BD as anti-seizure measures NPO for now as patient is restless and he is disoriented. Will give him iv fluids at 100 ml/hr. DVT prophylaxis with stockings. Avoid Heparin in view of subdural hemorrhage Prophylactic antibiotics to be started in view of the fracture  RS- Pt maintaining saturation on O2. Close watch of GCS. Pt might require intubation if GCS falls. Replace K - KCL 40 meq IV over 4 hrs in 250ml NS.  High risk for PE post op. Will need to watch.  Stop tramadol. High risk on head trauma.   PMH and Review of Systems:Nil HISTORY: Alleged H/O Fa

In [16]:
# Tokanizing sample into sentences
import nltk
sentences = nltk.sent_tokenize(sample)

In [ ]:
sentences

In [ ]:
# Tokenizing sentences into words
token_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [ ]:
# Tag words in sentences
pos_sentences = [nltk.pos_tag(sent) for sent in token_sentences]

In [ ]:
# Creating named entity chunks
chunked_sentences = nltk.ne_chunk_sents(pos_sentences, binary=True)

In [ ]:
for sent in chunked_sentences:
    for chunk in sent:
        if hasattr(chunk, 'label') and chunk.label() == 'NE':
            print(chunk)

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

ner_categories = defaultdict(int)
for sent in chunked_sentences:
    for chunk in sent:
        if hasattr(chunk, 'label'):
            ner_categories[chunk.label()] +=1
        
labels = list(ner_categories.keys())

values = [ner_categories.get(l) for l in labels]

plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
plt.show()

In [105]:
# Trying spacy for tokenization 
import spacy
from spacy.lang.en import English

nlp = spacy.load('en')
doc = nlp(sample2)

for ent in doc.ents:
    print(ent.label_, ent.text)

CARDINAL 12/15
ORG CT
ORG Neurosurgery
ORG CNS
ORG CT
ORG GCS
CARDINAL 12/15
ORG LOC
ORG GCS
ORG CT
PRODUCT Mannitol 100
PERSON Start Inj
QUANTITY 500 mg BD
ORG NPO
QUANTITY 100 ml
NORP Prophylactic
ORG GCS
ORG GCS
PERSON Replace K - KCL 40
ORG IV
GPE 250ml NS
GPE PE
ORG PMH
ORG Review of Systems


In [1]:
from quickumls import QuickUMLS
matcher = QuickUMLS(quickumls_fp='/home/dileep/QuickUMLS', threshold=0.4)

In [5]:
sample = 'Congestive Hert Failure, DBP, Pulse Rate, CVP, urine output, input output, GCS, Na+, K+, Cl-, CO2'
a = matcher.match(sample, best_match=True, ignore_syntax=False)

In [6]:
for i in range(len(a)):
    print(a[i][0]['ngram'], a[i][0]['semtypes'])

urine output {'T201'}
Pulse Rate {'T201'}
DBP {'T060'}
Congestive Hert Failure {'T047'}
CO2 {'T034'}
input output {'T201'}


In [58]:
import pyConTextNLP.pyConText as pyConText
import pyConTextNLP.itemData as itemData
import networkx as nx

In [60]:
reports = [
    """IMPRESSION: Evaluation limited by lack of IV contrast; however, no evidence of
      bowel obstruction or mass identified within the abdomen or pelvis. Non-specific interstitial opacities and bronchiectasis seen at the right
     base, suggestive of post-inflammatory changes.""",
    """IMPRESSION: Evidence of early pulmonary vascular congestion and interstitial edema. Probable scarring at the medial aspect of the right lung base, with no
     definite consolidation."""
    ,
    """IMPRESSION:
     
     1.  2.0 cm cyst of the right renal lower pole.  Otherwise, normal appearance
     of the right kidney with patent vasculature and no sonographic evidence of
     renal artery stenosis.
     2.  Surgically absent left kidney.""",
    """IMPRESSION:  No pneumothorax.""",
    """IMPRESSION: No definite pneumothorax"""
    """IMPRESSION:  New opacity at the left lower lobe consistent with pneumonia."""
]

In [119]:
modifiers = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.yml")
targets = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/utah_crit.yml")

/home/dileep/anaconda3/envs/cpcv/lib/python3.7/site-packages/pyConTextNLP/itemData.py:40: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  context_items =  [contextItem((d["Lex"],


In [158]:
def markup_sentence(s, modifiers, targets, prune_inactive=True):
    """
    """
    markup = pyConText.ConTextMarkup()
    markup.setRawText(s)
    markup.cleanText()
    markup.markItems(modifiers, mode="modifier")
    markup.markItems(targets, mode="target")
    markup.pruneMarks()
    markup.dropMarks('Exclusion')
    # apply modifiers to any targets within the modifiers scope
    markup.applyModifiers()
    markup.pruneSelfModifyingRelationships()
    if prune_inactive:
        markup.dropInactiveModifiers()
    return markup

In [170]:
print(reports[0])
context = pyConText.ConTextDocument()
report = reports[0]

IMPRESSION: Evaluation limited by lack of IV contrast; however, no evidence of
      bowel obstruction or mass identified within the abdomen or pelvis. Non-specific interstitial opacities and bronchiectasis seen at the right
     base, suggestive of post-inflammatory changes.


In [171]:
from textblob import TextBlob

blob = TextBlob(report.lower())
count = 0
rslts = []
for s in blob.sentences:
    m = markup_sentence(s.raw, modifiers=modifiers, targets=targets)
    rslts.append(m)

for r in rslts:
    context.addMarkup(r)

In [224]:
for node in rslts[0].nodes():
    print(node)

<id> 55825478535282733847928264409918309122 </id> <phrase> evaluation </phrase> <category> ['indication'] </category> 
<id> 55826186835055611371106350692834312962 </id> <phrase> no evidence of </phrase> <category> ['definite_negated_existence'] </category> 
<id> 55826722417434207798028483049938584322 </id> <phrase> however </phrase> <category> ['conj'] </category> 
<id> 55827001300566258008496812324643767042 </id> <phrase> bowel obstruction </phrase> <category> ['bowel_obstruction'] </category> 


In [194]:
markup = pyConText.ConTextMarkup()

True

In [203]:
markup.setRawText(reports[3].lower())
print(markup)
print(len(markup.getRawText()))

__________________________________________
rawText: impression:  no pneumothorax.
cleanedText: None
__________________________________________

29


In [205]:
markup.cleanText()

In [206]:
markup.getText()

'impression: no pneumothorax.'